## Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

## Read the CSV into DataFrame


In [2]:
# Load the data

df = pd.read_csv('lending_data.csv')
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


## Split the Data into Training and Testing


In [3]:
#Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

#Create our target
y= df['loan_status']

In [4]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [5]:
#check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [6]:
#Create X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)

## Data pre-processing

Scale the training and testing data using the StandardScaler from sklearn. Remember that when scaling the data, you only scale the features data (X_train and X_testing).

In [7]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [8]:
# Fit the Standard Scaler with the training data
#When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [9]:
#Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Simple Logistic Regression

In [10]:
#Generating the model
model = LogisticRegression(solver='lbfgs',random_state=1)
#Fitting the model with only training data
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [11]:
#Calculate the balanced accuracy score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [12]:
#Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [13]:
#Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the following steps: 

1.View the count of the target classes using Counter from the collections library.
    
    2. Usethe resampled data to train a logistic regression model.
    
    3. Calculate the balanced accuracy score from sklearn.metrics.
    
    4. Print the confusion matrix from sklearn.metrics.
    
    5. Generate a classification report using the imbalanced_classification_report from imbalanced_classification_report from imbalanced-learn.
    
Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests.

### Naive Random Oversampling

In [14]:
#Resample the training data with the RandomOversampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [15]:
#Train the Logistic Regression model using the rsampled data

model = LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [16]:
#Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [17]:
#Display the confusion matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  106, 18653]])

In [18]:
#Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



## SMOTE Oversampling

In [19]:
#Resample the training data with SMOTE


X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

In [20]:
#View the count of the target classes with counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
#Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [22]:
#Calculate the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9948279972279972

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  104, 18655]])

In [24]:
#Print the imbalanced classification reoprt
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



## Undersampling

In this section you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the following steps:

    1. View the count of the target classes using Counter from the collections library.
    2. Use the resampled data to train a logistic regression model.
    3. Calculate the balan ced accuracy score from sklearn.metrics 
    4. Display the confusion matrix from sklearn.metrics. 
    5 Generate a classification report using the imbalanced_classification_report from imbalanced-learn.

In [25]:
#Resample the data using the ClusterCentroids resampler

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)


In [26]:
#View the count of targer classes with counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [27]:
#Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
#Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9948279972279972

In [29]:
#Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  608,    17],
       [  102, 18657]])

In [30]:
#Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.97       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



## Combination (Over and Under) Sampling

In this section, you will test a combination over-and under-sampling algorithms above. You will resample the data using teh SMOTEENN algorithm and complete the following steps:

1. View the count of the target classes using Counter from the collections library.
2. Use the resamlped data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix form sklearn.metrics.
5. Generate a classification report using the imbalanced_classification_report from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
#Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)


In [32]:
#View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55290, 'low_risk': 55900})

In [33]:
#TRAIN THE LOGISTIC REGRESSION MODEL USING THE RESAMPLED DATA
model = LogisticRegression(solver='lbfgs',random_state=1)

In [34]:
#Fit the model
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
#Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9836813049736126

In [36]:
#Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  107, 18652]])

In [37]:
#Print the classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### Final Questions

Q.Which model had the best balanced accuracy score?
Answer: The Smote Oversampling algorithm and the Cluster Centroid algorithm both had the highest balanced accuracy score of approximately 0.9948.

Q.Which model had the best recall score?
Answer: All the models had an average recall score of 0.99. The random over sampler and the Smote oversampler both had the highest recall score of 1 for high risk.

Q.Which model had the best geometric mean score?
Answer: The best geometric mean score was 0.99 and the Random over sampler, the Smote oversampler and the Smoteenn all had this score.